**извлечение признаков из текста на естественном языке**

классификатор текстов

частотный анализ с очисткой стоп-слов (TF)

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
1196,"В Минске не проданы квартиры в пяти десятках сданных и строящихся ЖК\n\n27 ноября 2016 в 13:42\n\nИрина Осипова, REALTY.TUT.BY\n\nВласти неоднократно заявляли, что строительство в Минске будет огр...",realty
336,Геморрагическая лихорадка продолжает наступление на Оренбуржье В Оренбургской\nобласти продолжается высокий сезонный подъем заболеваемости геморрагической\nлихорадкой с почечным синдромом. На 15 н...,health


##  CountVectorizer + TF

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
stopwords= [
    'и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 
    'так', 'его', 'но',  'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 
    'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 
    'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или',  'ни', 'быть', 'был', 'него', 'до', 
    'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 
    'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем',
    'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 
    'ж', 'тогда', 'кто','этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 
    'этом', 'один', 'почти', 'мой', 'тем','чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем',
    'всех', 'никогда', 'можно', 'при', 'наконец', 'два','об', 'другой', 'хоть', 'после', 
    'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них','какая', 'много', 
    'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда',
    'лучше', 'чуть', 'том', 'нельзя', 'такой', 'им', 'более', 
    'всегда', 'конечно', 'всю', 'между']

In [8]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    # tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in stopwords ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов
    return ' '.join( [ t.strip() for t in tt if t ] )

In [9]:
# "ручная" очистка стоп-слов
tf = TfidfVectorizer(
    preprocessor=preprocessor,
    lowercase=False,
    stop_words=None,
    use_idf=False,
    norm='l2')

tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f93995b3bf8>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=False, vocabulary=None)

In [10]:
len(tf.vocabulary_)

44173

## формируем датасеты

In [11]:
X = tf.transform( data['text'] )
X.shape

(3196, 44173)

In [12]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [13]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

---

In [14]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [15]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9, random_state=get_seed() )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((319, 44173), (319,), (2877, 44173), (2877,))

## обучаем

In [16]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

## тестируем

In [17]:
o = clf.predict(X_train)

In [18]:
accuracy_score(y_train,o)

1.0

---

In [19]:
o = clf.predict(X_test)

In [20]:
accuracy_score(y_test,o)

0.754257907542579

In [21]:
# print( classification_report(y_test,o) )

---

In [22]:
# from matplotlib import pyplot as plt
# import itertools

# classes = sorted(labels.keys())
# cm = confusion_matrix(y_test,o)
# tick_marks = np.arange(len(classes))

# plt.figure(figsize=(10,9))

# plt.xticks(tick_marks, classes, rotation=45)
# plt.yticks(tick_marks, classes)

# thresh = cm.max() / 2.
# for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#     plt.text(j, i, format(cm[i, j], 'd'),
#              horizontalalignment="center",
#              color="white" if cm[i, j] > thresh else "black")

# plt.title('Confusion matrix')
# plt.tight_layout()
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
# plt.colorbar()

# plt.show()

---

In [23]:
o = clf.predict(X)

In [24]:
labels_inv = { labels[k]:k for k in labels}
# labels_inv

In [31]:
i = np.random.randint(len(data))
print('tag:',data.iloc[i,1])
print('predict:',labels_inv[o[i]])
print('- - - - - - - - - - - - - - - - - - \n')
print(data.iloc[i,0])


tag: auto
predict: auto
- - - - - - - - - - - - - - - - - - 

Новое поколение Q3 копания Audi строит, что вполне ожидаемо, на модульной платформе MQB концерна Volkswagen. Как утверждает издание Auto Bild, в Ингольштадте собираются воспользоваться всеми преимуществами платформы, в которые, помимо простоты разработки и возможности увеличить пространство салона, входит и шанс использовать трехцилиндровые двигатели.

Audi научились разговаривать со светофорами...

По данным издания, Audi Q3 получит как минимум один такой мотор, но пока неизвестно, какой именно – по предварительным сведениям, это будет либо бензиновый турбированный двигатель объемом 1,0 л, либо 1,4-литровый дизель. Впрочем, не исключено, что в Audi решатся предложить кроссоверу оба мотора.

Получит новая Audi Q3 и два варианта топового 2,5-литрового двигателя. Самая "заряженная" версия кроссовера под названием Q3 RS будет оснащаться мотором мощностью порядка 400 л.с., но в Audi впервые задумались о выпуске "промежуточной" S

---

In [26]:
# def preprocessor(text):
#     tt = [ t for t in text.split() if t ]
#     tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
#     tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
#     tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
#     tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
#     tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
#     return ' '.join( [ t.strip() for t in tt if t ] )
    
# from sklearn.feature_extraction.text import CountVectorizer
# stemmer = Stemmer('russian')
# analyzer = CountVectorizer().build_analyzer()
# def stemmed_words(doc): return [ Stemmer('russian').stemWord(w) for w in analyzer(doc) ]
# vect = CountVectorizer(lowercase=True,preprocessor=preprocessor,analyzer=stemmed_words)
# vect.fit( data['text'] )

# # размер словаря
# print(len(vect.vocabulary_))

# from sklearn.feature_extraction.text import TfidfTransformer
# tf = TfidfTransformer(use_idf=False, norm='l2')
# data_vect = vect.transform( data['text'] )
# tf.fit( data_vect )
